In [9]:
import pandas as pd  # таблицы
from collections import Counter  # счетчики
import datetime  # даты
import nltk # униграммы

dir = "D:\HSE\Pump_Dump\\"  # папка проекта

# фрейм ЦБ ММВБ
shares_pd = pd.read_csv(dir + "market\\moex_cb.csv", sep=",", encoding="utf-8")
shares_pd = shares_pd[(shares_pd["SUPERTYPE"] == "Акции")]
shares_pd.dropna(subset="INN", axis=0, inplace=True)
# удаляем строки без тикеров
shares_pd.dropna(subset="TRADE_CODE", axis=0, inplace=True)
# Тикеры к нижнему регистру
shares_pd["TRADE_CODE"] = shares_pd["TRADE_CODE"].str.lower()
shares_pd["LIST_SECTION"] = shares_pd["LIST_SECTION"].str.replace(" уровень", "")
shares_pd = shares_pd[["TRADE_CODE", "LIST_SECTION"]]

# загружаем список каналов
channel_path = dir + "TG\\" + "!channels" + ".csv"
channels_df = pd.read_csv(
    channel_path,
    encoding="utf-8",
    sep="|",
    keep_default_na=False,
)

In [10]:
# статистика по корпусу конкретного канала. Параметр - наименование канала и число подписчиков

def channel_load(channel):
    channel_path = dir + "TG\\" + channel + "_post.csv"
    # загрузка файла
    load_df = pd.read_csv(
        channel_path,
        encoding="utf-8",
        sep="|",
        keep_default_na=False,
    )
    return load_df

# статистическая обработка
def corps_stat(channel, df, subsc, period_date):
    # преобразуем поле с датой новости в тип ДАТА-ВРЕМЯ
    df["NEWS_DATE"] = pd.to_datetime(df["NEWS_DATE"], dayfirst=False, errors="ignore")
    # крайние N дней
    # period_date = 120
    to_date = pd.to_datetime("today").date()
    from_date = to_date - pd.Timedelta(days=period_date)
    from_date = from_date.strftime("%Y-%m-%d")
    to_date = to_date.strftime("%Y-%m-%d")
    # datetime.timedelta(days=30)
    df_period = df[(df["NEWS_DATE"] >= from_date) & (df["NEWS_DATE"] < to_date)]
    # распределение единичных постов по эшелонам тикеров (/уровень листинга)
    shares_list = df_period[
        (df_period["TICKERS"].str.count(",") < 1) & (df_period["TICKERS"] != "")
    ]["TICKERS"]
    shares_list_level = list()
    for share in shares_list:
        shares_list_level.append(
            shares_pd[(shares_pd["TRADE_CODE"] == share)]["LIST_SECTION"].values[0]
        )
    data = [
        {
            "CHANNEL": channel,
            "CHANNEL_SUBSC": subsc,
            "CHANNEL_ALL_COUNT": len(df),
            "CHANNEL_ALL_FROM": df["NEWS_DATE"].min().date(),
            "CHANNEL_ALL_TO": df["NEWS_DATE"].max().date(),
            "CHANNEL_PERIOD_FROM": df_period["NEWS_DATE"].min().date(),
            "CHANNEL_PERIOD_TO": df_period["NEWS_DATE"].max().date(),
            "CHANNEL_POST_DAY_COUNT": round(len(df_period) / period_date, 2),
            "CHANNEL_POST_LEN": int(df["NEWS_TEXT"].str.len().mean()),
            "CHANNEL_POST_TOKENS": len(",".join(df_period["WORDS"]).split(",")),
            "CHANNEL_PERIOD_COUNT": len(df_period),
            "CHANNEL_PERIOD_WITH_TICKERS": len(df_period[(df_period["TICKERS"] != "")]),
            "CHANNEL_PERIOD_ONE_TICKER": len(
                df_period[
                    (df_period["TICKERS"].str.count(",") < 1)
                    & (df_period["TICKERS"] != "")
                ]
            ),
            "CHANNEL_PERIOD_TOP_TICKERS": ",".join(
                [
                    word
                    for word, word_count in Counter(
                        list(
                            df_period[
                                (df_period["TICKERS"].str.count(",") < 1)
                                & (df_period["TICKERS"] != "")
                            ]["TICKERS"]
                        )
                    ).most_common(5)
                ]
            ),
            "CHANNEL_PERIOD_TOP_WORDS": ",".join(
                [
                    word
                    for word, word_count in Counter(
                        ",".join(df_period["WORDS"]).split(",")
                    ).most_common(10)
                ]
            ),
            "CHANNEL_PERIOD_MOOD": Counter(df_period["MOOD"].values).most_common(3),
            "CHANNEL_PERIOD_TICKERS_LEVELS": Counter(shares_list_level).most_common(3),
            "BIGRAMS": Counter(
                nltk.bigrams(",".join(df_period["WORDS"]).split(","))
            ).most_common(10),
            "TRIGRAMS": Counter(
                nltk.trigrams(",".join(df_period["WORDS"]).split(","))
            ).most_common(10),
        }
    ]
    i_df = pd.DataFrame.from_dict(data)
    return i_df

In [11]:
# фрейм для каждого канала
stat = pd.DataFrame()
# фрейм для общего корпуса (/итого)
load_df_big = pd.DataFrame()

k = 0
# print("Будет обработано: ",len(channels_df))
for channel in channels_df["CHANNEL"].values:
    k += 1
    # print(k, channel)
    # количество подписчиков
    subsc = channels_df[channels_df["CHANNEL"] == channel]["RELIABILITY"].values[0]
    # загружаем канал
    load_df = channel_load(channel)
    # отправляем его в статистическую обработку c количеством подписчиков и количеством дней
    stat_df = corps_stat(channel, load_df, subsc, 400)
    # сводим все каналы в один и статистический фрейм
    stat = pd.concat([stat, stat_df], ignore_index=False)
    # добавляем в общий корпус
    load_df_big = pd.concat([load_df_big, load_df], ignore_index=False)

# обрабатываем общий корпус
stat_big_df = corps_stat("All", load_df_big, "", 1000)

pd.concat([stat_big_df, stat], ignore_index=False).to_html(
     dir + "Answers\\" + "0_Channels.html"
)
# stat.to_html(dir + "Answers\\" + "0_Channels.html")

In [12]:
# load_df_big
# stat_big_df = corps_stat("All",load_df_big, "", 3650)
# stat_big_df